  # meta_step.04.GetFile_BlastAnnotation
  ## - author:Xiao Nan
  ## - begin date: 2024.3.4

In [42]:
import pandas as pd
import gzip
from Bio import SeqIO
import os
from tqdm.notebook import tqdm

## 拆分输入的表格

In [43]:
# set_option()
df_seqs_rep = pd.read_pickle('../blast_info/df_seqs_rep.pkl.gz')
df_seqs_rep.head(5)

,SeqRecord:rep,seq
rep,,
mG.blast.nt_163.fna.gz_LR862147.1_9597,"(M, A, D, V, V, A, S, G, V, L, S, S, L, V, D, ...",MADVVASGVLSSLVDRLRERITGEFGPVMDVGAELKKLEGTLSTIC...
mG.blast.nt_163.fna.gz_CP059603.1_2212,"(M, T, T, A, V, T, A, R, A, H, T, N, I, A, L, ...",MTTAVTARAHTNIALVKYWGKADQHLILPETSSLSLTLDQFYTDTT...
mG.blast.nt_163.fna.gz_CP059603.1_2212,"(M, T, T, A, V, T, A, R, A, H, T, N, I, A, L, ...",MTTAVTARAHTNIALVKYWGKADQHLILPETSSLSLTLDQFYTDTT...
mG.blast.nt_163.fna.gz_CP059603.1_2212,"(M, T, T, A, V, T, A, R, A, H, T, N, I, A, L, ...",MTTAVTARAHTNIALVKYWGKADQHLILPETSSLSLTLDQFYTDTT...
mG.blast.nt_163.fna.gz_CP059603.1_2212,"(M, T, T, A, V, T, A, R, A, H, T, N, I, A, L, ...",MTTAVTARAHTNIALVKYWGKADQHLILPETSSLSLTLDQFYTDTT...


In [44]:
for cas in df_seqs_rep["SeqRecord:rep"].tolist():
    print(cas.id)
    print(cas.description)
    print(cas.seq)
    break

mG.blast.nt_163.fna.gz_LR862147.1_9597
mG.blast.nt_163.fna.gz_LR862147.1_9597 assembly_id@mG.blast.nt_163.fna.gz;contig_id@LR862147.1;CDS_id@mG.blast.nt_163.fna.gz_LR862147.1_9597;CDS_start@10545777;CDS_end@10546967;CDS_predict_score@84.1;CDS_strand@+;CDS_phase@0;Predigal_info@ID=9597|partial=00|start_type=ATG|rbs_motif=None|rbs_spacer=None|gc_cont=0.460|conf=100.00|score=84.13|cscore=88.20|sscore=-4.07|rscore=0.00|uscore=-4.60|tscore=0.53;crispr_id@mG.blast.nt_163.fna.gz+LR862147.1+4;crispr_start@10553263;crispr_end@10553435;crispr_extend_start@10533263;crispr_extend_end@10573435;repeat_seq@GGCCCGGCCCGGCACGGCCC;n_copies@3;n_repeats@20;n_spacers@55;overlapped_base_count@1190 ID=1_9597;partial=00;start_type=ATG;rbs_motif=None;rbs_spacer=None;gc_cont=0.460;HEPN-1@[span=(219, 225),target=RQISDH];HEPN-count@1
MADVVASGVLSSLVDRLRERITGEFGPVMDVGAELKKLEGTLSTICDVLEDAEGRQVRERALRGWLGRLKDVAYDIDDLLDEHFAKNWKRSLTDPESSTTSWMKGTVSKVLSLPNSVMFQLKIAREIREIRERLDELAEERSKFHLQERCVVTCCRLENVVREETSSFVIESEVYGREEDKKK

In [45]:
df_seqs_rep.shape

(1084270, 2)

In [46]:
df_seqs_rep = df_seqs_rep.reset_index()

In [47]:
df_seqs_rep.drop_duplicates(subset='rep', inplace=True)
print("去重结束")

去重结束


In [48]:
df_seqs_rep.shape

(226983, 3)

In [49]:
226983/1000

226.983

## 进行拆分

In [53]:
# 创建临时储存的文件夹
temp = "../blast_info/fasta_split_1000"
os.makedirs(temp, exist_ok=True)

In [54]:
# 初始化计数器和批次号
counter = 0
batch_number = 1 # 用于给filename命名
contigs_in_batch = []  # 存储当前批次的contigs

for contig in tqdm(df_seqs_rep["SeqRecord:rep"].tolist()):
    # 添加contig到当前批次
    contigs_in_batch.append(contig)
    counter += 1
    # if counter == 5:
    #     print(contigs_in_batch)
    #     filename = f"{temp}/test.fasta"
    #     with gzip.open(filename, 'wt') if filename.endswith(".gz") else open(filename, 'wt') as handler_out:
    #         SeqIO.write(contigs_in_batch, handler_out, "fasta")
    #     break
    # 检查是否达到1000个序列或已经是最后一个序列
    if counter % 1000 == 0 or counter == len(df_seqs_rep["SeqRecord:rep"].tolist()):
        # 定义批次文件名
        filename = f"{temp}/batch_{batch_number}.fasta"
        with gzip.open(filename, 'wt') if filename.endswith(".gz") else open(filename, 'wt') as handler_out:
            SeqIO.write(contigs_in_batch, handler_out, "fasta")
        # 重置批次存储并更新批次号
        contigs_in_batch = []
        batch_number += 1

print("处理完成")

  0%|          | 0/226983 [00:00<?, ?it/s]

处理完成


In [50]:
# for contig in tqdm(df_seqs_rep["SeqRecord:rep"].tolist()):
#     # print(contig.id)
#     # print(contig.seq)
#     contig_id = contig.id
#     filename = f"{temp}/{contig_id}.fasta"
#     # 检查文件是否存在
#     if os.path.exists(filename):
#         # print(f"{filename} exists , skip ......")
#         continue
#     # print(filename)
#     handler_out = gzip.open(filename, 'wt') if filename.endswith(".gz") else open(filename, 'wt')
#     SeqIO.write(contig, handler_out, "fasta")



# handler.close()
# handler_out.close()
# print("ok")

In [51]:
# 顺序处理：通过简单地遍历 df_seqs_rep["SeqRecord:rep"].tolist()，每个蛋白序列按顺序被处理。这确保了不会跳过任何项，因为每个项都会按顺序遍历一次。